# Web Scraping METROCUADRADO

## Librerías

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

## Requerimiento a la pagína

## Selección de grilla

In [33]:
driver = webdriver.Chrome()
driver.get('https://www.metrocuadrado.com/venta/?search=form')
# Get the page source after interactions
page_source = driver.page_source
# Parse the page source with Beautiful Soup
soup = BeautifulSoup(page_source, 'lxml')

driver.quit()

In [4]:
house_grill = soup.find('ul', class_ = 'browse-results-list')

## Página inicial

In [39]:
name_houses_list = []
price_houses = []
area_houses = []
rooms_houses = []
baths_houses = []

for name_houses in house_grill:

    name = name_houses.find('h2', class_ = 'card-title').text
    name_houses_list.append(name)

    info = name_houses.find_all('p', class_ = 'card-subitem')
    
    price_houses.append(info[0].text)
    area_houses.append(info[1].text)
    rooms_houses.append(info[2].text)
    baths_houses.append(info[3].text)


df = pd.DataFrame(name_houses_list, columns=['Nombre_publicacion'])
df['Precio_inmueble'] = price_houses
df['Area_construida'] = area_houses
df['Habitaciones'] = rooms_houses
df['Banios'] = baths_houses
print(df.head(5))

                     Nombre_publicacion Precio_inmueble Area_construida  \
0   Casa en Venta, SIERRA NEVADA, Vill…    $700.000.000          350 m²   
1    Apartamento en Venta, CERROS DE S…  $1.400.000.000          225 m²   
2             Casa en Venta, chia, Chía  $2.800.000.000          420 m²   
3  Casa en Venta, condominio campestre…  $2.500.000.000          350 m²   
4  Casa en Venta, SOTILEZA, Bogotá D.C.  $2.500.000.000          642 m²   

  Habitaciones Banios  
0            5      5  
1            3      5  
2            5      5  
3            5      5  
4            3      4  


In [32]:
link_list = []

for link in house_grill.find_all('a', class_ = 'sc-bdVaJa ebNrSm', target = '_blank'):

    try:
        link_list.append('https://www.metrocuadrado.com' + link['href'])
    except:
        print('error')

(50, 5)

# Multiples páginas

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [4]:
driver = webdriver.Chrome()
driver.get('https://www.metrocuadrado.com/venta/?search=form')
# Get the page source after interactions
page_source = driver.page_source
# Parse the page source with Beautiful Soup
soup = BeautifulSoup(page_source, 'lxml')


df_final = pd.DataFrame()

num_page = 10

try:
    # Espera hasta que la página cargue completamente
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "browse-results-list")))

    # Ciclo para navegar a través de las páginas
    current_page = 1
    while True:
        
        # Extrae datos aquí, por ejemplo, nombres de productos
        
        df = pd.DataFrame()
        
        #elements = driver.find_elements(By.CLASS_NAME, "browse-results-list")
        house_grill = soup.find('ul', class_ = 'browse-results-list')
        name_houses_list = []
        price_houses = []
        area_houses = []
        rooms_houses = []
        baths_houses = []
        link_list = []

        for name_houses in house_grill:

            name = name_houses.find('h2', class_ = 'card-title').text
            name_houses_list.append(name)

            info = name_houses.find_all('p', class_ = 'card-subitem')
            
            price_houses.append(info[0].text)
            area_houses.append(info[1].text)
            rooms_houses.append(info[2].text)
            baths_houses.append(info[3].text)


        df = pd.DataFrame(name_houses_list, columns=['Nombre_publicacion'])
        df['Precio_inmueble'] = price_houses
        df['Area_construida'] = area_houses
        df['Habitaciones'] = rooms_houses
        df['Banios'] = baths_houses
        df['Pagina'] = current_page
        
        #Scraping link            

        for link in house_grill.find_all('a', class_ = 'sc-bdVaJa ebNrSm', target = '_blank'):

            try:
                link_list.append('https://www.metrocuadrado.com' + link['href'])
            except:
                print('Error')

        df['Link'] = list(dict.fromkeys(link_list))
        
        
        print("Página:",current_page)

        df_final = pd.concat([df_final, df], ignore_index=True)

        # Intenta encontrar el número de la siguiente página y hacer clic en él
        next_page = current_page + 1
        page_elements = driver.find_elements(By.CLASS_NAME, "pagination")

        find_element = driver.find_element(By.CLASS_NAME, "pagination")

        # Ejecutar JavaScript para hacer scroll hasta el elemento
        driver.execute_script("arguments[0].scrollIntoView(true);", find_element)        

        found_next_page = False 

        time.sleep(10)    

        if current_page == num_page:
            print("Total page scrap.")
            break   

        # # Iterar sobre cada elemento para imprimir su texto
        for page_element in page_elements:
            for page in page_element.find_elements(By.CLASS_NAME, 'page-link'):               
                if page.tag_name == 'a':
                    if page.text == str(next_page):                   
                        print(page)
                        page.click()
                        found_next_page = True
                        current_page = next_page
                        time.sleep(5)  # Espera para asegurar que la página se haya cargado completamente
                        break
        
        if not found_next_page:
            print("Reached last page.")
            break

# except Exception as e:
#      print(f"An error occurred: {e}")
finally:
    # Cierra el navegador
    driver.quit()
    

df_final.to_csv('metrocuadrado_principal.csv',index=False)

Página: 1
<selenium.webdriver.remote.webelement.WebElement (session="8c65c81f02a361ae2ce6bb90c99d4757", element="f.1886445005B919391D8FFF7869AC1B42.d.30916FD3909583224CBCE4DD96DCDE17.e.149")>
Página: 2
<selenium.webdriver.remote.webelement.WebElement (session="8c65c81f02a361ae2ce6bb90c99d4757", element="f.1886445005B919391D8FFF7869AC1B42.d.30916FD3909583224CBCE4DD96DCDE17.e.150")>
Página: 3
<selenium.webdriver.remote.webelement.WebElement (session="8c65c81f02a361ae2ce6bb90c99d4757", element="f.1886445005B919391D8FFF7869AC1B42.d.30916FD3909583224CBCE4DD96DCDE17.e.151")>
Página: 4
<selenium.webdriver.remote.webelement.WebElement (session="8c65c81f02a361ae2ce6bb90c99d4757", element="f.1886445005B919391D8FFF7869AC1B42.d.30916FD3909583224CBCE4DD96DCDE17.e.152")>
Página: 5
<selenium.webdriver.remote.webelement.WebElement (session="8c65c81f02a361ae2ce6bb90c99d4757", element="f.1886445005B919391D8FFF7869AC1B42.d.30916FD3909583224CBCE4DD96DCDE17.e.153")>
Página: 6
<selenium.webdriver.remote.web